In [35]:
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

In [36]:
# Function to create a test dataset from a portion of the training dataset
def create_test_dataset(src_folder, dest_folder, test_percentage):
    os.makedirs(dest_folder, exist_ok=True)
    classes = os.listdir(src_folder)

    for class_name in classes:
        class_path = os.path.join(src_folder, class_name)
        dest_class_path = os.path.join(dest_folder, class_name)
        os.makedirs(dest_class_path, exist_ok=True)

        images = os.listdir(class_path)
        num_images_to_copy = int(len(images) * test_percentage)
        test_images = random.sample(images, num_images_to_copy)

        for image in test_images:
            src_image_path = os.path.join(class_path, image)
            dest_image_path = os.path.join(dest_class_path, image)
            shutil.copy(src_image_path, dest_image_path)

In [37]:
# Paths and parameters
combined_train_dataset_path = r'C:\Users\DELL\Desktop\dataset\training_set'
test_dataset_path = r'C:\Users\DELL\Desktop\dataset\test_set'
img_width, img_height = 224, 224
batch_size = 32
test_percentage = 0.2

In [38]:
# Create test dataset from training dataset
create_test_dataset(combined_train_dataset_path, test_dataset_path, test_percentage)

In [39]:
# Create data generators for training and validation with a split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [41]:
# Combine all training datasets into one training generator
train_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    combined_train_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 327 images belonging to 4 classes.


In [42]:
# Create data generator for validation
validation_generator = datagen.flow_from_directory(
    combined_train_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 64 images belonging to 4 classes.


In [43]:
# Build a simple CNN model using VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

In [44]:
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))  # 4 classes: copper, brass, still, non-metal


In [45]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [46]:
# Train the model
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
11/11 [==============================] - 90s 8s/step - loss: 49.3511 - accuracy: 0.4832 - val_loss: 0.9713 - val_accuracy: 0.5781
Epoch 2/10
11/11 [==============================] - 107s 10s/step - loss: 8.8645 - accuracy: 0.7064 - val_loss: 1.1738 - val_accuracy: 0.4688
Epoch 3/10
11/11 [==============================] - 101s 9s/step - loss: 3.9638 - accuracy: 0.8563 - val_loss: 0.7661 - val_accuracy: 0.6719
Epoch 4/10
11/11 [==============================] - 103s 9s/step - loss: 3.0330 - accuracy: 0.8502 - val_loss: 0.9017 - val_accuracy: 0.6562
Epoch 5/10
11/11 [==============================] - 103s 9s/step - loss: 2.3947 - accuracy: 0.8777 - val_loss: 0.9476 - val_accuracy: 0.5625
Epoch 6/10
11/11 [==============================] - 101s 9s/step - loss: 1.8448 - accuracy: 0.9174 - val_loss: 0.8736 - val_accuracy: 0.7188
Epoch 7/10
11/11 [==============================] - 102s 9s/step - loss: 1.7944 - accuracy: 0.9327 - val_loss: 0.8772 - val_accuracy: 0.7031
Epoch 8/10
1

In [47]:
# Evaluate the model on the test set
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    test_dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 55 images belonging to 4 classes.


In [48]:
test_eval = model.evaluate(test_generator)
print("Test Accuracy:", test_eval[1])

2/2 [==============================] - 11s 5s/step - loss: 2.1674e-08 - accuracy: 1.0000
Test Accuracy: 1.0
